### Setup env
This is notebook specific setup, my module path is different on my vm so this is a workaround

In [1]:
import sys
import os

path = os.path.abspath(os.path.join('..'))
if path not in sys.path:
    sys.path.append(path)

### Load Dataset
You can change the type of motion by changing the filepath
Dataset right now just repeats the same motion 100 times

In [2]:
from diffusion.data_loaders.motion_dataset import MotionDataset
dataset = MotionDataset("data/motions/humanoid3d_cartwheel.txt")
len(dataset), dataset[0], dataset[0].trajectories.shape

Tmp angle [0.0, 0.0, 0.85536, 0.9966429999999997, -0.0070009999999999795, 0.08157, 0.0005729999999999971, 0.042303731260289315, -0.056088768155961526, -0.01172717680484046, -0.014103614145860938, 0.2358842735659614, 0.37124889801787253, -0.6111023347690597, -0.09268300376873025, -0.09541896434572254, 0.585361, 0.1699928747321186, 0.08652758875118252, 0.354108626550405, 0.160215, -0.2285399691330798, -0.39445967594673703, -0.1178224382194308, -0.369571, 0.20448116583595066, -0.12115992907931128, 0.07892319943485762, 0.3736623102073797, -0.010008232584494297, 0.30603690929303384, -0.364281, -0.13425257761871864, -0.004787718949892447, 0.0010873114649849894] 35


(160,
 Batch(trajectories=tensor([[ 0.0000e+00,  0.0000e+00,  8.5536e-01,  ..., -1.3425e-01,
          -4.7877e-03,  1.0873e-03],
         [ 3.6500e-03, -6.2240e-03,  8.5632e-01,  ..., -1.3036e-01,
           1.4385e-02,  1.5935e-02],
         [ 6.4000e-03, -1.2205e-02,  8.5748e-01,  ..., -1.3041e-01,
           3.6052e-02,  3.1853e-02],
         ...,
         [-2.4664e-01,  2.0540e+00,  8.4651e-01,  ...,  5.9186e-02,
          -2.0091e-01,  2.3318e-01],
         [-2.4351e-01,  2.0658e+00,  8.4672e-01,  ...,  7.3301e-02,
          -2.0666e-01,  2.1504e-01],
         [-2.3938e-01,  2.0782e+00,  8.4690e-01,  ...,  8.5008e-02,
          -2.1333e-01,  1.9921e-01]]), conditions={0: tensor([ 0.0000e+00,  0.0000e+00,  8.5536e-01,  9.9664e-01, -7.0010e-03,
          8.1570e-02,  5.7300e-04,  4.2304e-02, -5.6089e-02, -1.1727e-02,
         -1.4104e-02,  2.3588e-01,  3.7125e-01, -6.1110e-01, -9.2683e-02,
         -9.5419e-02,  5.8536e-01,  1.6999e-01,  8.6528e-02,  3.5411e-01,
          1.6022e-0

In [2]:
dataset[0]

NameError: name 'dataset' is not defined

### Setup Model
Configure your experiment name and savepaths here, they will all be stored under the logs folder later on

In [3]:
import os
import torch
from diffusion.diffuser.utils import Trainer as dTrainer, Config as dConfig

exp_name = "test-cartwheel-shuffled-seq"
savepath = f'/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/{exp_name}'
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

if not os.path.exists(savepath):
    os.makedirs(savepath)
    os.makedirs(os.path.join(savepath, 'sampled_motions'))

In [5]:
from diffusion.diffuser.models.temporal import TemporalUnet 

horizon = dataset[0].trajectories.shape[0]
transition_dim = dataset[0].trajectories.shape[1]

model_config = dConfig(
    TemporalUnet,
    savepath=(savepath, 'model_config.pkl'),
    horizon=horizon,
    transition_dim=transition_dim,
    cond_dim=transition_dim,
    device=device,
)
model = model_config()


[utils/config ] Config: <class 'diffusion.diffuser.models.temporal.TemporalUnet'>
    cond_dim: 35
    horizon: 160
    transition_dim: 35

[ utils/config ] Saved config to: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-cartwheel-shuffled-seq/model_config.pkl

[ models/temporal ] Channel dimensions: [(35, 32), (32, 64), (64, 128), (128, 256)]
[(35, 32), (32, 64), (64, 128), (128, 256)]


In [6]:
from diffusion.diffuser.models.diffusion import GaussianDiffusion

# model params, I am only using the very basic ones, some params are for conditioning
n_timesteps = 1000
loss_type = 'l2'
clip_denoised = False
predict_epsilon = False
action_weight = 5
loss_weights = None
loss_discount = 1

diffusion_config = dConfig(
    GaussianDiffusion,
    savepath=(savepath, 'diffusion_config.pkl'),
    horizon=horizon,
    transition_dim=transition_dim,
    n_timesteps=n_timesteps,
    loss_type=loss_type,
    clip_denoised=clip_denoised,
    predict_epsilon=predict_epsilon,
    # loss weighting
    action_weight=action_weight,
    loss_weights=loss_weights,
    loss_discount=loss_discount,
    device=device,
)

diffusion = diffusion_config(model)


[utils/config ] Config: <class 'diffusion.diffuser.models.diffusion.GaussianDiffusion'>
    action_weight: 5
    clip_denoised: False
    horizon: 160
    loss_discount: 1
    loss_type: l2
    loss_weights: None
    n_timesteps: 1000
    predict_epsilon: False
    transition_dim: 35

[ utils/config ] Saved config to: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-cartwheel-shuffled-seq/diffusion_config.pkl



### Setup Training

In [7]:
learning_rate = 2e-4
gradient_accumulate_every = 2
ema_decay = 0.995
sample_freq = 2000
save_freq = 2000
n_train_steps = 1e5
n_saves = 5
save_parallel = False
bucket = None
n_reference = 8
train_batch_size = 32

trainer_config = dConfig(
    dTrainer,
    savepath=(savepath, 'trainer_config.pkl'),
    train_batch_size=train_batch_size,
    train_lr=learning_rate,
    gradient_accumulate_every=gradient_accumulate_every,
    ema_decay=ema_decay,
    sample_freq=sample_freq,
    save_freq=save_freq,
    label_freq=int(n_train_steps // n_saves),
    save_parallel=save_parallel,
    results_folder=savepath,
    bucket=bucket,
    n_reference=n_reference,
)

trainer = trainer_config(diffusion, dataset, renderer=None)


[utils/config ] Config: <class 'diffusion.diffuser.utils.training.Trainer'>
    bucket: None
    ema_decay: 0.995
    gradient_accumulate_every: 2
    label_freq: 20000
    n_reference: 8
    results_folder: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-cartwheel-shuffled-seq
    sample_freq: 2000
    save_freq: 2000
    save_parallel: False
    train_batch_size: 32
    train_lr: 0.0002

[ utils/config ] Saved config to: /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-cartwheel-shuffled-seq/trainer_config.pkl



### Test if model and training loop works

In [8]:
import torch
from diffuser.utils import batchify

tunet = model
tunet.to(device)
test_data = dataset[0]
batch = batchify(test_data)
t = torch.randint(0, 1000, (1,), device=device).long().to(device)
res = tunet(batch.trajectories, cond=batch.conditions, time=t, verbose=True)
res, res.shape

x.shape torch.Size([1, 160, 35])
x.shape torch.Size([1, 35, 160])
x.shape torch.Size([1, 32, 160])
1
xfinal.shape torch.Size([1, 32, 80])
x.shape torch.Size([1, 64, 80])
2
xfinal.shape torch.Size([1, 64, 40])
x.shape torch.Size([1, 128, 40])
3
xfinal.shape torch.Size([1, 128, 20])
x.shape torch.Size([1, 256, 20])
4
xfinal.shape torch.Size([1, 256, 20])
xt1.shape torch.Size([1, 256, 20])
xt2.shape torch.Size([1, 256, 20])
xt3.shape torch.Size([1, 256, 20])
pop.shape torch.Size([1, 256, 20])
pop.shape torch.Size([1, 128, 40])
pop.shape torch.Size([1, 64, 80])


(tensor([[[ 0.0978,  0.0336,  0.0114,  ...,  0.2216, -0.1019,  0.0748],
          [-0.0201,  0.0227,  0.1396,  ...,  0.2933, -0.4068,  0.4115],
          [ 0.0071,  0.2636,  0.0058,  ..., -0.0106, -0.2060, -0.0839],
          ...,
          [ 0.0388, -0.0390,  0.2037,  ...,  0.1892, -0.1845,  0.3176],
          [ 0.2324,  0.1583, -0.1952,  ...,  0.1847, -0.3760,  0.2647],
          [ 0.1079, -0.0967, -0.0466,  ...,  0.2292, -0.1339,  0.1934]]],
        device='cuda:0', grad_fn=<PermuteBackward0>),
 torch.Size([1, 160, 35]))

In [9]:
from diffuser.utils import report_parameters, batchify

report_parameters(model)

print('Testing forward...', end=' ', flush=True)
x = dataset[0]
batch = batchify(x)
loss, _ = diffusion.loss(batch.trajectories, {})
loss.backward()
print('✓')

[ utils/arrays ] Total parameters: 3.96 M
         downs.3.0.blocks.1.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         downs.3.1.blocks.0.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         downs.3.1.blocks.1.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         ups.0.0.blocks.0.block.0.weight: 327.68 k | Conv1d(512, 128, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block1.blocks.0.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block1.blocks.1.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block2.blocks.0.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         mid_block2.blocks.1.block.0.weight: 327.68 k | Conv1d(256, 256, kernel_size=(5,), stride=(1,), padding=(2,))
         downs.3.0.b

### Train Model
It took me 80s to run 1 epoch and results were pretty good from just 1 epoch

In [15]:
n_steps_per_epoch = 1000
n_epochs = int(n_train_steps // n_steps_per_epoch)
print(n_epochs)
n_epochs = 2

for i in range(n_epochs):
    print(f'Epoch {i} / {n_epochs} | {savepath}')
    trainer.train(n_train_steps=n_steps_per_epoch)

trainer.save(n_epochs)

100
Epoch 0 / 2 | /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-cartwheel-shuffled-seq
1000:   0.0080 | a0_loss:   0.0035 | t:   0.6308
1100:   0.0061 | a0_loss:   0.0034 | t:   9.1947
1200:   0.0068 | a0_loss:   0.0031 | t:   8.6260
1300:   0.0095 | a0_loss:   0.0036 | t:   8.5765
1400:   0.0071 | a0_loss:   0.0035 | t:   9.0732
1500:   0.0071 | a0_loss:   0.0030 | t:   8.9400
1600:   0.0040 | a0_loss:   0.0023 | t:   8.6111
1700:   0.0085 | a0_loss:   0.0028 | t:   8.6322
1800:   0.0055 | a0_loss:   0.0025 | t:   8.5434
1900:   0.0039 | a0_loss:   0.0026 | t:   8.9542
Epoch 1 / 2 | /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-cartwheel-shuffled-seq
[ utils/training ] Saved model to /home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/test-cartwheel-shuffled-seq/state_0.pt
2000:   0.0026 | a0_loss:   0.0012 | t:   0.2277
2100:   0.0059 | a0_loss:   0.0027 | t:   8.5767
2200:   0.0062 | a0_loss:   0.0034 | t:   8.7216
2300:   0.0079 | a0_loss:   0.0037 | t:   8.8342
2400:   0

### (Alternatively) load a checkpoint

In [23]:
# from diffusion.diffuser.utils import load_diffusion
# diffusion_experiment = load_diffusion(
#     "/home/kenji/Fyp/DeepMimic_mujoco/diffusion/logs/diffuser", dataset=dataset, epoch=1)

# renderer = diffusion_experiment.renderer
# model = diffusion_experiment.trainer.ema_model

# Optionally load a checkpoint
trainer.load(1)
model = trainer.ema_model

### Sample from model
This is the plain configuration, nothing special about it

In [13]:
from diffuser.utils import batchify
test = dataset[0]
batch = batchify(test)
sample = diffusion.p_sample_loop(batch.trajectories.shape, batch.conditions)


                                                                                                    
1 / 1000 [                                                            ]   0% | 4.6 Hz
t : 999 | vmax : 0.0 | vmin : 0.0
                                                                                                    
                                                                                                    
2 / 1000 [                                                            ]   0% | 8.6 Hz
t : 998 | vmax : 0.0 | vmin : 0.0
                                                                                                    
                                                                                                    
3 / 1000 [                                                            ]   0% | 12.4 Hz
t : 997 | vmax : 0.0 | vmin : 0.0
                                                                                                    
                                

### Save results to logs

In [14]:
import numpy as np


def save_motions(sample, output_dir, filename="motion.npy"):
    filepath = os.path.join(output_dir, filename)
    pos_data = sample.trajectories.squeeze(0).cpu().numpy()
    np.save(filepath, pos_data)
    print(f"Motion saved as {filename}")


save_motions(sample, f"{savepath}/sampled_motions", filename="cartwheel-shuffled-motion.npy")

Motion saved as cartwheel-shuffled-motion.npy
